In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ape import networks, Contract
import pandas as pd
import altair as alt

from helpers.coingecko import get_votium_prices
from helpers.votium import CVX_FEE
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [7]:
_, _, badger_price, fxs_price = get_votium_prices()
data = [["${:.2f}".format(badger_price), "${:.2f}".format(fxs_price)]]
headers = ["BADGER price", "FXS price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,BADGER price,FXS price
0,$2.86,$10.04


In [12]:
# Incentive program. Starts on 9th Jan, last incentive post on the 26th Jan
WEEKLY = 1500
TOTAL_INCENTIVES = 1500 * 4

In [22]:
# FXS claims & pendants. Directly querying from farm to avoid estimations based on pool capture etc
# There was already a claim on the 3rd Jan, so 3rd Jan till the round of the 9th Jan is considered un-incentivesed
farm = Contract("0x5a92EF27f4baA7C766aee6d751f754EBdEBd9fae")
dao_private_vault = "0xa895B89D74a6BC23a284a0526e123ea776674cF5"
CLAIM_BLOCK = 16478637 # one before: https://etherscan.io/tx/0x6b08c3763f4d38993a0eaca06558184549eeab55f5c51e71ee774e44955a7107
fxs_earned = farm.earned(dao_private_vault, block_identifier=CLAIM_BLOCK)[0] / 1e18
fxs_pendant = farm.earned(dao_private_vault)[0] / 1e18
total_fxs_earned = (fxs_earned + fxs_pendant) * (1 - CVX_FEE)

In [23]:
# Cost vs earnings in dollar terms
earnings = total_fxs_earned * fxs_price
costs = TOTAL_INCENTIVES * badger_price

In [24]:
# Basic groupe bar chart
df = pd.DataFrame([[earnings, 'R'], 
                   [costs, 'C']],
                  columns=['$ Value', 'P&L'])

chart = alt.Chart(df, width=400).mark_bar().encode(
   x=alt.X('P&L'),
   y=alt.Y('$ Value'),
   color=alt.Color('P&L', scale=alt.Scale(range=['#FF5733', '#33FF47']))
)

chart.display() 

alt.Chart(...)